<a href="https://colab.research.google.com/github/NITHIN-KESHAV/DATA-228-Group-Project/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gdown


## Working on chunk data( 10,000 rows )

In [0]:
import gdown

file_id = "11CHzQfUChpnG1EGRWQXPs4VYIcmtVQZj"
download_url = f"https://drive.google.com/uc?id={file_id}"

output = 'yourfile.zip'
gdown.download(download_url, output, quiet=False)



In [0]:
import os

# Checking the size of the file to ensure it was downloaded completely
file_size = os.path.getsize('yourfile.zip')
print(f"File size: {file_size / (1024 * 1024):.2f} MB")


In [0]:
# Checking the first few bytes of the file to see if it's a valid zip file
with open('yourfile.zip', 'rb') as file:
    header = file.read(4)
    print(header)


In [0]:
with gzip.open('yourfile.zip', 'rb') as f:
    chunk_size = 100000  
    for chunk in pd.read_csv(f, chunksize=chunk_size, sep='\t'):  
        print(chunk.head())
        break



In [0]:
with gzip.open('yourfile.zip', 'rb') as f:
    for i, line in enumerate(f):
        print(line)
        if i == 20:  # Stoping the loop after the first 20 lines
            break



In [0]:
import gzip
import pandas as pd

# Opening the gzip file and reading the data as JSON lines
with gzip.open('yourfile.zip', 'rb') as f:
    # Reading the JSON lines into a pandas DataFrame
    df = pd.read_json(f, lines=True)

# Displaying the first few rows to get a better view of the data
print(df.head(5))


In [0]:
import gzip
import pandas as pd

# Defining the chunk size
chunk_size = 10000  

# Opening the gzip file and processing it in chunks
with gzip.open('yourfile.zip', 'rb') as f:
    # Reading the file in chunks and processing each chunk
    for chunk in pd.read_json(f, lines=True, chunksize=chunk_size):
        print(chunk.head())  
        break


In [0]:
print(chunk.columns)  
print(chunk.dtypes)   


In [0]:
print(chunk.describe())  


In [0]:
print(chunk.isnull().sum())  


In [0]:
chunk['review_length'] = chunk['reviewText'].apply(lambda x: len(str(x).split()))
print(chunk[['reviewText', 'review_length']].head()) 


In [0]:
from google.colab import files
files.download('processed_chunk.csv')


## Steps for topic modelling for chunks


In [0]:
!pip install gensim scikit-learn


In [0]:
import pandas as pd

df = pd.read_csv('/processed_chunk.csv')

print(df.head())


## Preprocessing the Text for Topic Modeling:

In [0]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# Downloading the stopwords
nltk.download('stopwords')

# Preprocessing function: clean and tokenize text
def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text))  # Removing special characters
    text = re.sub(r'\s+', ' ', text)  # Removing extra spaces
    tokens = text.lower().split()  # Lowercasing and tokenization
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Removing stopwords
    return tokens

# Applying preprocessing to the 'reviewText' column
df['tokens'] = df['reviewText'].apply(preprocess_text)

# Checking the tokenized reviews
print(df[['reviewText', 'tokens']].head())


##  Creating Dictionary and Corpus:

In [0]:
from gensim import corpora

# Creating a dictionary representation of the documents
dictionary = corpora.Dictionary(df['tokens'])

# Creating a bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Showing the sample corpus
print(corpus[:1])


## Applying LDA for Topic Modeling:

In [0]:
import gensim

# Setting the number of topics
num_topics = 10

# Running LDA using gensim
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            num_topics=num_topics,
                                            id2word=dictionary,
                                            passes=10)

# Printing the topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)


### Visualizing the Topics

In [0]:
!pip install pyLDAvis


In [0]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

pyLDAvis.display(vis_data)
